In [2]:
import pandas as pd
import gensim.downloader as api
from scipy.spatial.distance import cosine
import numpy as np
import re

In [5]:
ai_completed_file_path = r"C:\Users\bened\OneDrive\Documents\Businesses\Relationship Predicting\Tesco Clubcards\4 - Processed Data Files\Categorisations\categorised_items_270524_1951.xlsx"
category_file_path = r"C:\Users\bened\OneDrive\Documents\Businesses\Relationship Predicting\Tesco Clubcards\2 - Training Data\Categorised Fine Tuning.xlsx"

ai_completed_df = pd.read_excel(ai_completed_file_path)

#Open list of categories, flavours & tags & turn into a list
category_df = pd.read_excel(category_file_path, sheet_name="LLM Fine Tuning")

# Select all columns except "Line Item"
columns_to_consider = category_df.drop(columns=['Line Item'])

# Unstack the DataFrame, drop NaN values, and get unique values
category_list = columns_to_consider.unstack().dropna().unique().tolist()

#Get list of columns we care about 
target_list = ai_completed_df.columns[1:].tolist()

In [12]:
# Common substitutions dictionary
common_subs = {
    "&": "and",
    "/": " or ",
    "yogurt": "yoghurt",
    "confectionary": "confectionery",
    "squash": "cordial",
    "flavor": "flavour",
    "color": "colour",
}

def apply_subs(s):
    s = s.lower()
    for k, v in common_subs.items():
        s = s.replace(k, v)
    s = re.sub(r'\s+', ' ', s)  # Replace multiple spaces with a single space
    return s



canned fooandd
canned and preserved goods


In [46]:
# Load pre-trained Word2Vec model
# model = api.load("word2vec-google-news-300")

def embeddings_similarity(str1, str2):
    vec1 = np.mean([model[word] for word in str1.split() if word in model], axis=0)
    vec2 = np.mean([model[word] for word in str2.split() if word in model], axis=0)
    return 1 - cosine(vec1, vec2)

# Example usage
str1 = apply_subs("Canned Food")
str2 = apply_subs("Canned & Preserved Goods")
print(embeddings_similarity(str1, str2))


0.6676396341964741


In [49]:
# Example usage
str1 = apply_subs("Canned Food")
str2 = apply_subs("Tinned Food")
print(embeddings_similarity(str1, str2))

0.798123953523817


In [44]:
import numpy as np
from scipy.spatial.distance import cosine
import gensim.downloader as api

# Load pre-trained Word2Vec model
model = api.load("word2vec-google-news-300")

# Function to get word embedding, handling missing words
def get_word_embedding(word):
    if word in model:
        return model[word]
    else:
        return np.zeros(model.vector_size)

# Function to calculate similarity
def embeddings_similarity(vec1, vec2):
    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0  # Avoid dividing by zero
    return 1 - cosine(vec1, vec2)

# Function to find best match
def find_best_match(mis_categorized, actual_categories):
    max_similarity = -1
    best_match = None
    mis_categorized_embedding = np.mean([get_word_embedding(word) for word in mis_categorized.split()], axis=0)
    
    print(f"Mis-categorized embedding for '{mis_categorized}': {mis_categorized_embedding}")

    for category in actual_categories:
        category_embedding = np.mean([get_word_embedding(word) for word in category.split()], axis=0)
        similarity = embeddings_similarity(mis_categorized_embedding, category_embedding)
        print(f"Similarity with '{category}': {similarity}")
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = category
    
    return best_match, max_similarity

# Example usage
mis_categorized = "Canned Food"
best_match, similarity_score = find_best_match(mis_categorized, category_list)
print(f"Best match: {best_match} with similarity score of {similarity_score:.4f}")


Mis-categorized embedding for 'Canned Food': [-0.18209839 -0.10150146 -0.13671875  0.12158203 -0.01867676  0.14855957
 -0.00268555 -0.12768555  0.10668945  0.09082031  0.00878906  0.06738281
  0.07543945 -0.07019043 -0.22558594  0.05337524  0.00390625  0.03442383
 -0.19433594 -0.01367188  0.34960938  0.33203125  0.3828125   0.05990601
 -0.015625   -0.13232422  0.04492188  0.11682129  0.23291016  0.03036499
 -0.3388672  -0.10656738  0.21191406 -0.00732422 -0.1075058  -0.04031372
 -0.1730957  -0.10864258 -0.16516113  0.0300293  -0.09350586 -0.13134766
 -0.02270508  0.12414551 -0.09936523 -0.0032959  -0.3203125   0.11096191
  0.0559082   0.22949219 -0.4013672  -0.14916992 -0.11791992 -0.08032227
  0.06311035  0.02807617  0.04943848 -0.10876465  0.13220215 -0.07458496
 -0.25195312  0.09802246 -0.40039062  0.19268799  0.21142578 -0.14257812
 -0.00073242 -0.1628418   0.11108398 -0.3623047   0.02124023 -0.30371094
 -0.00524902 -0.06030273 -0.17822266 -0.20028591 -0.12036133 -0.27197266
  0.19

In [42]:
actual_category_embeddings# Function to get the mean vector of a string using Word2Vec
def get_mean_vector(model, words):
    words = words.split()
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:
        print(f"Warning: None of the words in '{words}' were found in the model!")
        return np.zeros(model.vector_size)
    mean_vector = np.mean(word_vectors, axis=0)
    return mean_vector

# Encode all actual categories
actual_category_embeddings = [get_mean_vector(model, category) for category in category_list]

def find_best_match(mis_categorized_word, actual_category_list, actual_category_embeddings):
    # Encode the mis-categorized word
    mis_categorized_embedding = get_mean_vector(model, mis_categorized_word)
    if np.all(mis_categorized_embedding == 0):
        print(f"Warning: None of the words in '{mis_categorized_word}' were found in the model!")
    
    # Compute cosine similarity between the mis-categorized word and all actual categories
    similarities = [1 - cosine(mis_categorized_embedding, category_embedding) for category_embedding in actual_category_embeddings]
    
    # Find the index of the highest similarity score
    best_match_index = np.argmax(similarities)
    
    # Return the category with the highest similarity score
    return actual_category_list[best_match_index], similarities[best_match_index]

# Example usage
mis_categorized_word = "Canned Food"
best_match, similarity_score = find_best_match(mis_categorized_word, category_list, actual_category_embeddings)
print(f"Best match: {best_match} with similarity score of {similarity_score:.4f}")

Best match: Uncategorised with similarity score of 1.0000
